In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import gsw

In [2]:
from utils import *

In [3]:
output_path="/home1/datahome/jmartine/models/nemo/tests/MY_ICE_CANAL/EXP00/forcing/"

In [4]:
ngrid_x = 501
ngrid_y = 253
ngrid_z = 101

In [5]:
δx=2
δy=2
δz=1
max_depth=800
x=np.linspace(0,ngrid_x*δx,ngrid_x)
y=np.linspace(0,ngrid_y*δy,ngrid_y)

depth,_ = compute_depth_prof(ngrid_z,max_depth)

In [6]:
Ys,Zs = np.meshgrid(y,depth) 

In [7]:
from utils import *
y_0 = (δy*ngrid_y)/2
w  = 100/2.5
ice_profile=smooth_tanh(y,w=w,x_0=y_0)

In [8]:
t = 365

In [9]:
β = -(1/2)
γ =  np.linspace(-np.pi,np.pi,t)
season = 1/2 + np.cos(β +γ)/2

In [10]:
start = -100
end   = 600

In [11]:
L = start+(end-start)*season

In [12]:
Lf = 100

H1=75
dh=75

In [13]:
horizontal_depend_tanh = (smooth_tanh(Ys,Lf,L)) * smooth_tanh(-Zs,dh,-H1)

In [14]:
horizontal_depend_tanh.shape

(365, 101, 253)

In [15]:
salt   = - 3/2 * horizontal_depend_tanh[0]
salt_0 = 35
ft_s= gsw.t_freezing(salt_0+salt,0,0)


In [16]:
temp_atm=xr.open_dataset("./temp2m_climatology_daily.nc")

min_temp = temp_atm.t2m.min().values-273.15
max_temp = temp_atm.t2m.max().values-273.15

In [18]:
temp = horizontal_depend_tanh[:,0,:] * (min_temp-max_temp) + max_temp

In [24]:
radiation=xr.open_dataset("./SWR_and_LWR_climatology_daily.nc")

LWR=radiation.msdwlwrf.isel(dayofyear=slice(0,365)).values
SWR=radiation.msdwswrf.isel(dayofyear=slice(0,365)).values
TEMP = temp_atm.t2m.isel(dayofyear=slice(0,365)).values

In [27]:
# Zonal wind [m/s]
ds_u10=np.zeros((t,ngrid_y,ngrid_x))

# Meridional wind [m/s]
ds_v10=np.zeros((t,ngrid_y,ngrid_x))

# Shortwave radiation [W/m2]
# ds_rsw=np.ones((1,103,201))*10.
ds_rsw=np.ones((t,ngrid_y,ngrid_x)) * SWR[:,None,None]

# Longwave radiation [W/m2]
# ds_rlw=np.ones((1,103,201))*170.
ds_rlw=np.ones((t,ngrid_y,ngrid_x))*LWR[:,None,None]

# Air temperature [K]
ds_t10=np.ones((t,ngrid_y,ngrid_x))*TEMP[:,None,None]
# ds_t10=np.ones((t,ngrid_y,ngrid_x))*#273.15 + temp[:,:,None]

# Air humidity [-]
ds_q10=np.ones((t,ngrid_y,ngrid_x))*0.00130145

# precipitation [kg/m2/s]
ds_pre=np.zeros((t,ngrid_y,ngrid_x))

# Snow [kg/m2/s]
ds_sno=np.zeros((t,ngrid_y,ngrid_x))

# Sea level pressure [Pa]
ds_slp=np.ones((t,ngrid_y,ngrid_x))*100000.

In [28]:
ds_for=xr.Dataset()

ds_for['u10']=(('time','y','x'),ds_u10)
ds_for['v10']=(('time','y','x'),ds_v10)
ds_for['rsw']=(('time','y','x'),ds_rsw)
ds_for['rlw']=(('time','y','x'),ds_rlw)
ds_for['t10']=(('time','y','x'),ds_t10)
ds_for['q10']=(('time','y','x'),ds_q10)
ds_for['pre']=(('time','y','x'),ds_pre)
ds_for['sno']=(('time','y','x'),ds_sno)
ds_for['slp']=(('time','y','x'),ds_slp)

In [32]:
ds_for.to_netcdf(output_path+'Channel-Surf-Forcing_new_ALL_seasonal_new_grid_HR_HTR_out_boundaries_april_center_no_atm_temp_moving.nc',unlimited_dims={'time':True})